In [1]:
import mlflow
import pandas as pd
import mlflow.sklearn
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import pandas as pd
import re
import string
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import numpy as np

In [2]:
import nltk
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to C:\Users\Aditya
[nltk_data]     Arya\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to C:\Users\Aditya
[nltk_data]     Arya\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [3]:
df = pd.read_csv('IMDB.csv')
df = df.sample(500)
df.to_csv('data.csv', index=False)
df.head()

,review,sentiment
970,I saw this back in '94 when it was finally rel...,negative
353,I thought this had the right blend of characte...,positive
454,After watching about half of this movie I noti...,negative
233,I saw this movie with very low expectations. I...,negative
983,There is no way to put into words just how bad...,negative


In [4]:
# data preprocessing

# Define text preprocessing functions
def lemmatization(text):
    """Lemmatize the text."""
    lemmatizer = WordNetLemmatizer()
    text = text.split()
    text = [lemmatizer.lemmatize(word) for word in text]
    return " ".join(text)

def remove_stop_words(text):
    """Remove stop words from the text."""
    stop_words = set(stopwords.words("english"))
    text = [word for word in str(text).split() if word not in stop_words]
    return " ".join(text)

def removing_numbers(text):
    """Remove numbers from the text."""
    text = ''.join([char for char in text if not char.isdigit()])
    return text

def lower_case(text):
    """Convert text to lower case."""
    text = text.split()
    text = [word.lower() for word in text]
    return " ".join(text)

def removing_punctuations(text):
    """Remove punctuations from the text."""
    text = re.sub('[%s]' % re.escape(string.punctuation), ' ', text)
    text = text.replace('؛', "")
    text = re.sub('\s+', ' ', text).strip()
    return text

def removing_urls(text):
    """Remove URLs from the text."""
    url_pattern = re.compile(r'https?://\S+|www\.\S+')
    return url_pattern.sub(r'', text)

def normalize_text(df):
    """Normalize the text data."""
    try:
        df['review'] = df['review'].apply(lower_case)
        df['review'] = df['review'].apply(remove_stop_words)
        df['review'] = df['review'].apply(removing_numbers)
        df['review'] = df['review'].apply(removing_punctuations)
        df['review'] = df['review'].apply(removing_urls)
        df['review'] = df['review'].apply(lemmatization)
        return df
    except Exception as e:
        print(f'Error during text normalization: {e}')
        raise

In [5]:
df = normalize_text(df)
df.head()

,review,sentiment
970,saw back finally released apparently orion pic...,negative
353,thought right blend character plot futuristic ...,positive
454,watching half movie noticed something peculiar...,negative
233,saw movie low expectation know lot sure going ...,negative
983,way put word bad shapeless paceless laughless ...,negative


In [6]:
df['sentiment'].value_counts()

sentiment
negative    258
positive    242
Name: count, dtype: int64

In [7]:
x = df['sentiment'].isin(['positive','negative'])
df = df[x]

In [8]:
df['sentiment'] = df['sentiment'].map({'positive':1, 'negative':0})
df.head()

,review,sentiment
970,saw back finally released apparently orion pic...,0
353,thought right blend character plot futuristic ...,1
454,watching half movie noticed something peculiar...,0
233,saw movie low expectation know lot sure going ...,0
983,way put word bad shapeless paceless laughless ...,0


In [9]:
df.isnull().sum()

review       0
sentiment    0
dtype: int64

In [15]:
vectorizer = CountVectorizer(max_features=50)
X = vectorizer.fit_transform(df['review'])
y = df['sentiment']

In [14]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [12]:
import dagshub

mlflow.set_tracking_uri('https://dagshub.com/AdArya125/MLOPS-Capstone-Project.mlflow')
dagshub.init(repo_owner='AdArya125', repo_name='MLOPS-Capstone-Project', mlflow=True)

# mlflow.set_experiment("Logistic Regression Baseline")
mlflow.set_experiment("Logistic Regression Baseline")


Accessing as AdArya125

Initialized MLflow to track repo "AdArya125/MLOPS-Capstone-Project"

Repository AdArya125/MLOPS-Capstone-Project initialized!

<Experiment: artifact_location='mlflow-artifacts:/a3f3514f961f4d9d98b71375fa6b5192', creation_time=1749137269050, experiment_id='0', last_update_time=1749137269050, lifecycle_stage='active', name='Logistic Regression Baseline', tags={}>

In [16]:
import mlflow
import logging
import os
import time
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# Configure logging
logging.basicConfig(level=logging.INFO, format="%(asctime)s - %(levelname)s - %(message)s")

logging.info("Starting MLflow run...")

with mlflow.start_run():
    start_time = time.time()
    
    try:
        logging.info("Logging preprocessing parameters...")
        mlflow.log_param("vectorizer", "Bag of Words")
        mlflow.log_param("num_features", 50)
        mlflow.log_param("test_size", 0.2)

        logging.info("Initializing Logistic Regression model...")
        model = LogisticRegression(max_iter=1000)  # Increase max_iter to prevent non-convergence issues

        logging.info("Fitting the model...")
        model.fit(X_train, y_train)
        logging.info("Model training complete.")

        logging.info("Logging model parameters...")
        mlflow.log_param("model", "Logistic Regression")

        logging.info("Making predictions...")
        y_pred = model.predict(X_test)

        logging.info("Calculating evaluation metrics...")
        accuracy = accuracy_score(y_test, y_pred)
        precision = precision_score(y_test, y_pred)
        recall = recall_score(y_test, y_pred)
        f1 = f1_score(y_test, y_pred)

        logging.info("Logging evaluation metrics...")
        mlflow.log_metric("accuracy", accuracy)
        mlflow.log_metric("precision", precision)
        mlflow.log_metric("recall", recall)
        mlflow.log_metric("f1_score", f1)

        logging.info("Saving and logging the model...")
        mlflow.sklearn.log_model(model, "model")

        # Log execution time
        end_time = time.time()
        logging.info(f"Model training and logging completed in {end_time - start_time:.2f} seconds.")

        # Save and log the notebook
        # notebook_path = "exp1_baseline_model.ipynb"
        # logging.info("Executing Jupyter Notebook. This may take a while...")
        # os.system(f"jupyter nbconvert --to notebook --execute --inplace {notebook_path}")
        # mlflow.log_artifact(notebook_path)

        # logging.info("Notebook execution and logging complete.")

        # Print the results for verification
        logging.info(f"Accuracy: {accuracy}")
        logging.info(f"Precision: {precision}")
        logging.info(f"Recall: {recall}")
        logging.info(f"F1 Score: {f1}")

    except Exception as e:
        logging.error(f"An error occurred: {e}", exc_info=True)


2025-06-05 21:06:21,625 - INFO - Starting MLflow run...
2025-06-05 21:06:22,060 - INFO - Logging preprocessing parameters...
2025-06-05 21:06:23,323 - INFO - Initializing Logistic Regression model...
2025-06-05 21:06:23,324 - INFO - Fitting the model...
2025-06-05 21:06:23,358 - INFO - Model training complete.
2025-06-05 21:06:23,359 - INFO - Logging model parameters...
2025-06-05 21:06:23,797 - INFO - Making predictions...
2025-06-05 21:06:23,798 - INFO - Calculating evaluation metrics...
2025-06-05 21:06:23,815 - INFO - Logging evaluation metrics...
2025-06-05 21:06:25,456 - INFO - Saving and logging the model...
2025/06/05 21:06:30 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
2025-06-05 21:06:34,288 - INFO - Model training and logging completed in 12.23 seconds.
2025-06-05 21:06:34,291 - INFO - Accuracy: 0.57
2025-06-05 21:06:34,296 - INFO - Precision: 0

🏃 View run bouncy-dolphin-389 at: https://dagshub.com/AdArya125/MLOPS-Capstone-Project.mlflow/#/experiments/0/runs/fa46860a2ac24c04a07d588301613a7f
🧪 View experiment at: https://dagshub.com/AdArya125/MLOPS-Capstone-Project.mlflow/#/experiments/0
